# Load lib

In [1]:
import os
import numpy as np
import pandas as pd
import tifffile
import matplotlib.pyplot as plt
from skimage import filters, measure, morphology, feature, segmentation
import bioio
import bioio_tifffile
from scipy import ndimage as ndi
from skimage.morphology import white_tophat, ball
import ast
import re
from tqdm import tqdm
import ipywidgets as widgets
from IPython.display import display
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from skimage.measure import marching_cubes
from scipy.stats import norm



# Variables

In [3]:
#image_path="/mnt/external.data/MeisterLab/Dario/SDC1/1273/20240813_e/N2V_sdc1_dpy27_mSG_emr1_mCh/denoised/20240813_1273_E_late_15um_08_n2v.tif",
image_path="/mnt/external.data/MeisterLab/Dario/DPY27/1268/20240808_e/N2V_dpy27_mSG_emr1_mCh/denoised/20240808_1268_E_late_15um_08_n2v.tif",
nuclei_mask_path="/mnt/external.data/MeisterLab/mvolosko/image_project/sdc1v1/SDC1/1273/20240813_3d/segmentation/SDC1_3d_20240813_1273_E_late_15um_08_t00.tif"
nuclei_csv_path="/mnt/external.data/MeisterLab/mvolosko/image_project/sdc1v1/SDC1/1273/20240813_3d/nuclei/SDC1_3d_20240813_1273_E_late_15um_08.csv",
mask_path="/mnt/external.data/MeisterLab/mvolosko/image_project/sdc1v1/SDC1/1273/20240813_3d/segmentation/SDC1_3d_20240813_1273_E_late_15um_08_t00.tif"    

In [1]:
from aicsimageio import AICSImage
from pathlib import Path

img_path = Path("/mnt/external.data/MeisterLab/Dario/SDC1/1273/20241010_e_tl/N2V_dpy27_mSG_emr1_mCh/denoised/20241010_1273_E_early_3h_5min_5um_03_n2v.tif")
img = AICSImage(str(img_path))

# Print axes and shape
print("Axes:", img.dims.order)   # e.g., 'TCZYX'
print("Shape:", img.shape)       # tuple of dimensions


Axes: TCZYX
Shape: (36, 2, 18, 1282, 1222)


In [3]:
import bioio
bio = bioio.BioImage("/mnt/external.data/MeisterLab/mvolosko/image_project/SDC1/1273/20241108_e_hs/spots/raw_images_timelapse/20241107_1273_E_30minHS_3h_5min_5um_n2v_t00.ome.tif")
img = bio.get_image_data("ZYX", C=0)
print(img.shape)


(26, 1024, 1006)


In [5]:

def split_5d_tiff_to_timepoints(input_path: Path, output_dir: Path) -> None:
    """
    Splits a 5D (T, C, Z, Y, X) TIFF into one TIFF per timepoint.
    Outputs to output_dir/raw_images_timelapse/t{tt}.tif with tt=00,01,...
    """
    input_dir = Path(input_path)
    raw_root = Path(output_dir) / "raw_images_timelapse"
    raw_root.mkdir(parents=True, exist_ok=True)

    ext = {".tif", ".tiff"}
    for img_path in sorted(input_dir.iterdir()):
        if img_path.suffix.lower() not in ext or img_path.stem.endswith('_max'):
            continue
        sample = img_path.stem
        out_sub = raw_root / sample
        out_sub.mkdir(exist_ok=True)

        # lazy open
        img = AICSImage(str(img_path))
        n_time, n_chan, n_z, n_y, n_x = img.shape  # (T, C, Z, Y, X)

        for t in range(n_time):
            # grab channels+Z for this timepoint as (C,Z,Y,X)
            czyx = img.get_image_data("CZYX", T=t)
            # tifffile wants (Z, Y, X, C)
            zyx_c = np.moveaxis(czyx, 0, -1)

            # write one multi-page TIFF
            out_path = out_sub / f"t{t:02d}.tif"
            tifffile.imwrite(
                str(out_path),
                zyx_c,
                photometric="minisblack",
                metadata={"axes": "ZYCX"},
            )

        img.close()

In [7]:
split_5d_tiff_to_timepoints(input_path='/mnt/external.data/MeisterLab/Dario/DPY27/1268/20241010_e_tl/N2V_dpy27_mSG_emr1_mCh/denoised/', 
output_dir= "/mnt/external.data/MeisterLab/mvolosko/image_project/DPY27/1268/20241010_e_tl/")


KeyboardInterrupt: 

# Spot detection + nuclei filtering

In [4]:
def detect_spots_MPHD(image_path, spot_channel=1, 
                     sigma=1.3, h_percentile=85,  
                     min_distance=2,              
                     min_volume=10,                
                     max_volume=100, 
                     intensity_percentile=80,     
                     peak_percentile=70,
                     nuclei_mask_path=None):         
    """
    MPHD spot detection with adaptive h-dome transformation.
    Parameters:
    - image_path: Path to the 3D TIFF image.
    - spot_channel: Channel index for spot detection.
    - sigma: Gaussian smoothing parameter.
    - h_percentile: Percentile for adaptive h value.
    - min_distance: Minimum distance between detected spots.
    - min_volume: Minimum volume of detected spots.
    - max_volume: Maximum volume of detected spots.
    - intensity_percentile: Percentile for intensity filtering.
    - peak_percentile: Percentile for peak thresholding.
    - nuclei_mask_path: Optional path to nuclei mask for final filtering
    """
    # Load image
    img_4d = bioio.BioImage(image_path, reader=bioio_tifffile.Reader)
    img_raw = img_4d.get_image_data("ZYX", T=0, C=spot_channel).astype(np.float32)
    
    # Normalize to [0,1] based on image statistics
    img = (img_raw - img_raw.min()) / (img_raw.max() - img_raw.min() + 1e-6)
    
    # Load nuclei mask if provided
    nuclei_mask = None
    if nuclei_mask_path is not None:
        mask_4d = bioio.BioImage(nuclei_mask_path, reader=bioio_tifffile.Reader)
        nuclei_mask = mask_4d.get_image_data("ZYX", T=0, C=0).astype(bool)
        if nuclei_mask.shape != img.shape:
            raise ValueError("Nuclei mask shape does not match image shape.")

    # Gaussian smoothing with 3D kernel
    smoothed = filters.gaussian(img, sigma=sigma)
    
    # Calculate adaptive h value
    non_zero = smoothed[smoothed > 0]
    if len(non_zero) == 0:
        raise ValueError("No signal detected after smoothing!")
        
    h = np.percentile(non_zero, h_percentile)
    
    # 3D h-dome transformation
    seed = np.clip(smoothed - h, 0, None)
    footprint = morphology.ball(1)
    reconstructed = morphology.reconstruction(seed, smoothed, method='dilation', footprint=footprint)
    hdome = smoothed - reconstructed
    
    # Adaptive peak threshold
    non_zero_hdome = hdome[hdome > 0]
    if len(non_zero_hdome) > 0:
        peak_thresh = np.percentile(non_zero_hdome, peak_percentile)
    else:
        raise ValueError("H-dome transformation failed - check h value!")
    
    # Detect regional maxima
    coordinates = feature.peak_local_max(
        hdome,
        min_distance=min_distance,
        threshold_abs=peak_thresh,
        exclude_border=False
    )

    # Create markers
    mask = np.zeros_like(hdome, dtype=bool)
    mask[tuple(coordinates.T)] = True
    markers = measure.label(mask)

    # Watershed segmentation
    if hdome.max() > 0:
        mask_thresh = hdome > filters.threshold_otsu(hdome)
    else:
        mask_thresh = np.zeros_like(hdome, dtype=bool)
    
    labels = segmentation.watershed(-hdome, markers, mask=mask_thresh)

    # Measure properties
    props = measure.regionprops_table(
        labels,
        intensity_image=img,
        properties=('label', 'centroid', 'area', 'max_intensity')
    )
    
    df = pd.DataFrame(props).rename(columns={
        'centroid-0': 'centroid_z', 'centroid-1': 'centroid_y', 'centroid-2': 'centroid_x',
        'area': 'volume',
        'max_intensity': 'intensity'
    })
    
    # Add peak coordinates
    peak_df = pd.DataFrame(coordinates, columns=['peak_z', 'peak_y', 'peak_x'])
    peak_df['label'] = markers[tuple(coordinates.T)]
    df = df.merge(peak_df, on='label', how='left')

    # Intensity filtering
    if len(img[img > 0]) > 0:
        intensity_thresh = np.percentile(img[img > 0], intensity_percentile)
        df = df[df['intensity'] > intensity_thresh]
    
    # Volume filteringindices=range(11, 14)
    df = df[df['volume'].between(min_volume, max_volume)]

    # Apply nuclei mask filtering * 0.8
    if nuclei_mask is not None:
        print("Applying nuclei mask filtering to final spots")
        # Convert float coordinates to integer indices
        z_coords = np.round(df['centroid_z']).astype(int).clip(0, nuclei_mask.shape[0]-1)
        y_coords = np.round(df['centroid_y']).astype(int).clip(0, nuclei_mask.shape[1]-1)
        x_coords = np.round(df['centroid_x']).astype(int).clip(0, nuclei_mask.shape[2]-1)
        
        # Check if spots are in masked regions
        in_nuclei = nuclei_mask[z_coords, y_coords, x_coords]
        df = df[in_nuclei]
        print(f"After nuclei filtering: {len(df)} spots remaining")

    return df, img, hdome, labels

def visualize_results(img, df, hdome, labels):
    """Visualization for MPHD results"""
    max_proj = np.max(img, axis=0)
    hdome_proj = np.max(hdome, axis=0)
    # mid_z = img.shape[0] // 2
    
    # # Filter by PEAK Z-coordinate
    # df_mid = df[df['peak_z'] == mid_z]

    plt.figure(figsize=(10, 5))
    
    plt.subplot(1, 2, 1)
    plt.imshow(max_proj, cmap='gray', vmax=np.percentile(max_proj, 99.5))
    plt.scatter(df['peak_x'], df['peak_y'], s=20, facecolors='none', 
                edgecolors='red', linewidth=0.5)
    plt.title(f"Detected spots: {len(df)}")
    
    plt.subplot(1, 2, 2)
    plt.imshow(hdome_proj, cmap='viridis')
    plt.title("H-Dome projection")


# Count spots per nuclei

In [ ]:
    def load_nuclei_data(nuclei_csv_path):
        """Load nuclei data with array columns parsing"""
        df = pd.read_csv(nuclei_csv_path)
        
        def parse_array(arr_str):
            try:
                cleaned = re.sub(r'[^\d\.,\[\]]', '', str(arr_str))
                cleaned = re.sub(r',+', ',', cleaned)
                return np.array(ast.literal_eval(cleaned))
            except (ValueError, SyntaxError):
                return np.array([])

        # Process array columns
        array_cols = ['zproj_spots', 'zproj_nuclei', 'intensity_dist', 'intensity_dist_spots']
        for col in array_cols:
            if col in df.columns:
                df[col] = df[col].apply(parse_array)
        
        # Removed incorrect bounding box tuple processing
        return df

    def count_spots_in_nuclei(spots_df, mask_path, nuclei_df):
        """Enhanced spot counting with physical coordinate conversion"""
        # Load mask with pixel size handling
        mask_img = bioio.BioImage(mask_path, reader=bioio_tifffile.Reader)
        mask = mask_img.get_image_data("ZYX", T=0, C=0)
        
        # Handle physical pixel sizes
        try:
            pixel_sizes = mask_img.physical_pixel_sizes
        except AttributeError:
            pixel_sizes = (1.0, 1.0, 1.0)
        
        # Convert coordinates with pixel size adjustment
        spots_df = spots_df.copy()
        axis_map = {'z': 0, 'y': 1, 'x': 2}
        
        for axis in ['z', 'y', 'x']:
            ps = pixel_sizes[axis_map[axis]]
            dim = mask.shape[axis_map[axis]]
            ps_val = ps if ps is not None and ps > 0 else 1.0

            col_name = f'centroid_{axis}'
            if col_name not in spots_df.columns:
                raise ValueError(f"Missing required column: {col_name}")
                
            spots_df[f'{axis}_idx'] = (
                spots_df[f'centroid_{axis}']
                .div(ps_val)
                .round()
                .astype(int)
                .clip(0, dim - 1)
            )
        
        # Get labels for each spot
        mask = mask.astype(np.uint32)
        spots_df['nucleus_label'] = mask[
            spots_df['z_idx'].values,
            spots_df['y_idx'].values,
            spots_df['x_idx'].values
        ]
        
        # Ensure label column exists before merging
        if 'nucleus_label' not in spots_df.columns:
            raise RuntimeError("Failed to assign nucleus labels to spots")
        
        # Count spots per nucleus with proper merging
        spot_counts = (
            spots_df[spots_df['nucleus_label'] > 0]
            .groupby('nucleus_label', observed=True)
            .size()
            .rename('spot_count')
            .astype(np.uint32)
        )
        
        # Merge with nuclei data
        nuclei_df = nuclei_df.merge(
            spot_counts,
            left_on='label',
            right_index=True,
            how='left'
        ).fillna({'spot_count': 0})
        
        
        return spots_df, nuclei_df

    def calculate_spatial_metrics(nuclei_df):
        """Calculate advanced spatial metrics"""
        # 3D density metrics
        if 'volume' in nuclei_df.columns:
            nuclei_df['spots_per_volume'] = nuclei_df['spot_count'] / (nuclei_df['volume'] + 1e-6)
        
        # Surface area approximation
        if all(col in nuclei_df.columns for col in ['major_axis_length', 'solidity', 'anisotropy']):
            a = nuclei_df['major_axis_length']/2
            b = a * nuclei_df['solidity']
            c = b * nuclei_df['anisotropy']
            nuclei_df['surface_area'] = 4 * np.pi * (((a*b)**1.6 + (a*c)**1.6 + (b*c)**1.6)/3)**(1/1.6)
        
        return nuclei_df

    def visualize_distribution(nuclei_df):
        """Enhanced visualization with multiple plots and normal distribution fit"""
        fig = plt.figure(figsize=(16, 12))
        
        # Plot 1: Volume vs Spot Count
        ax1 = fig.add_subplot(221)
        if 'volume' in nuclei_df.columns and 'spot_count' in nuclei_df.columns:
            color_data = nuclei_df['anisotropy'] if 'anisotropy' in nuclei_df.columns else 'blue'
            cmap = 'viridis' if 'anisotropy' in nuclei_df.columns else None
            scatter = ax1.scatter(
                nuclei_df['volume'],
                nuclei_df['spot_count'],
                c=color_data,
                cmap=cmap,
                alpha=0.6)
            ax1.set_xlabel('Nuclear Volume (pixels³)')
            ax1.set_ylabel('Spot count')
            # if 'anisotropy' in nuclei_df.columns:
            #     plt.colorbar(scatter, ax=ax1, label='Anisotropy')

        # Plot 2: Spots per Volume with Normal Fit
        ax2 = fig.add_subplot(222)
        if 'spots_per_volume' in nuclei_df.columns:
            spots_per_vol = nuclei_df['spots_per_volume'].dropna()
            if not spots_per_vol.empty:
                # Histogram with counts
                n, bins, patches = ax2.hist(spots_per_vol, bins=30, alpha=0.6, color='green')
                
                # Calculate normal distribution parameters
                mu, sigma = spots_per_vol.mean(), spots_per_vol.std()
                x = np.linspace(spots_per_vol.min(), spots_per_vol.max(), 100)
                
                # Scale normal curve to match histogram
                y = norm.pdf(x, mu, sigma) * len(spots_per_vol) * (bins[1] - bins[0])
                ax2.plot(x, y, 'r--', linewidth=2)
                ax2.set_xlabel('Spots per volume (spots/pixel³)')
                ax2.set_ylabel('Count')
                ax2.set_title(f'Normal distribution fit\nμ={mu:.2f}, σ={sigma:.2f}')

        # Plot 3: Spot Count Distribution
        ax3 = fig.add_subplot(223)
        if 'spot_count' in nuclei_df.columns:
            counts = nuclei_df['spot_count'].value_counts().sort_index()
            counts.plot(kind='bar', ax=ax3, color='teal')
            ax3.set_xlabel('Spot count per nucleus')
            ax3.set_ylabel('Frequency')
            ax3.set_title('Distribution of spot counts')

        # Plot 4: Strain Comparison or Density Plot
        ax4 = fig.add_subplot(224)
        if 'strain' in nuclei_df.columns and 'spots_per_volume' in nuclei_df.columns:
            # Boxplot by strain
            strains = nuclei_df['strain'].unique()
            data = [nuclei_df[nuclei_df['strain'] == s]['spots_per_volume'].dropna() for s in strains]
            ax4.boxplot(data, labels=strains, showmeans=True)
            ax4.set_ylabel('Spots per Volume (spots/μm³)')
            ax4.set_title('Distribution by Strain')
        elif 'spots_per_volume' in nuclei_df.columns:
            # Density plot if no strain information
            spots_per_vol = nuclei_df['spots_per_volume'].dropna()
            if not spots_per_vol.empty:
                ax4.hist(spots_per_vol, bins=30, density=True, alpha=0.6, color='purple')
                ax4.set_xlabel('Spots per volume (spots/pixel³)')
                ax4.set_ylabel('Density')
                ax4.set_title('Probability density')

        plt.tight_layout()
        plt.show()

    def analyze_spots(spots_df, nuclei_csv_path, mask_path):
        """Integrated analysis pipeline"""
        print("Loading nuclei data...")
        nuclei_df = load_nuclei_data(nuclei_csv_path)
        
        print("Mapping spots to nuclei...")
        with tqdm(total=3) as pbar:
            spots_mapped, nuclei_counts = count_spots_in_nuclei(spots_df, mask_path, nuclei_df)
            pbar.update(1)
            
            # Calculate aspect ratios for each plane
            nuclei_counts['xy_ratio'] = nuclei_counts.apply(
                lambda row: min(row['bb_dimX'], row['bb_dimY']) / max(row['bb_dimX'], row['bb_dimY']), axis=1)
            nuclei_counts['xz_ratio'] = nuclei_counts.apply(
                lambda row: min(row['bb_dimX'], row['bb_dimZ']) / max(row['bb_dimX'], row['bb_dimZ']), axis=1)
            nuclei_counts['yz_ratio'] = nuclei_counts.apply(
                lambda row: min(row['bb_dimY'], row['bb_dimZ']) / max(row['bb_dimY'], row['bb_dimZ']), axis=1)
            
            # Filter nuclei based on aspect ratios
            filtered_nuclei = nuclei_counts[
                (nuclei_counts['xy_ratio'] >= 0.5) &
                (nuclei_counts['xz_ratio'] >= 0.2) &
                (nuclei_counts['yz_ratio'] >= 0.2)
            ].copy()
            
            nuclei_metrics = calculate_spatial_metrics(filtered_nuclei)
            pbar.update(1)
            
            print("Generating visualizations...")
            visualize_distribution(nuclei_metrics)
            pbar.update(1)
        
        return spots_mapped, nuclei_metrics

    def explore_z_stack(img, spots_df, nuclei_mask=None):
        """
        Interactive 3D explorer for image z-stack with spots overlay.
        :param img: 3D image array (Z, Y, X) to explore.
        :param spots_df: DataFrame with at least 'z_idx', 'y_idx', and 'x_idx' columns.
        :param nuclei_mask: (Optional) 3D nuclei mask array to overlay boundaries.
        """
        z_max = img.shape[0] - 1

        def plot_slice(z_idx):
            plt.figure(figsize=(8, 6))
            slice_img = img[z_idx]
            plt.imshow(slice_img, cmap='gray', vmax=np.percentile(slice_img, 99))
            # Overlay spots which have the same z-slice (or close to it)
            idx = spots_df['z_idx'] == z_idx
            #idx = spots_df['peak_z'] == z_idx
            if idx.sum() > 0:
                plt.scatter(spots_df.loc[idx, 'x_idx'],
                            spots_df.loc[idx, 'y_idx'],
                            s=40, facecolors='none', edgecolors='r', linewidth=1.2)
            # Optionally overlay nuclei boundaries if provided
            if nuclei_mask is not None:
                # Get nuclei mask slice; overlay boundaries
                from skimage import measure
                slice_mask = nuclei_mask[z_idx].astype(np.uint8)
                contours = measure.find_contours(slice_mask, 0.5)
                for contour in contours:
                    plt.plot(contour[:, 1], contour[:, 0], color='lime', linewidth=1)
            plt.title(f"Z slice: {z_idx}   Spots: {idx.sum()}")
            plt.axis('off')
            plt.show()

        # Create an interactive widget to scroll through the z-stack.
        slider = widgets.IntSlider(min=0, max=z_max, step=1, value=z_max // 2, description='Z Slice:')
        widgets.interact(plot_slice, z_idx=slider)


# RUN

In [ ]:
if __name__ == "__main__":
    #First run your spot detection
    df, img, hdome, labels = detect_spots_MPHD(
        image_path=image_path[0],
        spot_channel=1,
        sigma=1.3,               # Start with smaller sigma
        h_percentile=55,         # Lower h-percentile
        peak_percentile=70,      # Lower peak threshold
        min_distance=1,          # Reduced separation
        min_volume=5,            # Smaller volume
        max_volume=100,
        intensity_percentile=80, # More lenient intensity cutoff
        nuclei_mask_path=nuclei_mask_path 
    )
    visualize_results(img, df, hdome, labels)

    # Use the results to analyze spots in the nuclei
    spots_mapped, nuclei_analysis = analyze_spots(
        spots_df=df,
        nuclei_csv_path="/mnt/external.data/MeisterLab/mvolosko/image_project/sdc1v1/SDC1/1273/20240813_3d/nuclei/SDC1_3d_20240813_1273_E_late_15um_08.csv",
        mask_path=nuclei_mask_path 
    )


    # Save the spots with nucleus labels
    spots_mapped = spots_mapped.merge(
        nuclei_analysis[['label', 'anisotropy', 'experiment', 'strain']],
        left_on='nucleus_label',
        right_on='label',
        how='left'
    ).drop(columns=['label_y']).rename(columns={'label_x': 'label'})

    # explore_z_stack(img, spots_mapped, nuclei_mask=None)  
    
    # Save results
    spots_mapped.to_csv("/mnt/external.data/MeisterLab/mvolosko/image_project/sdc1v1/spots_with_nucleus_labels.csv", index=False)
    nuclei_analysis.to_csv("/mnt/external.data/MeisterLab/mvolosko/image_project/sdc1v1/nuclei_with_spot_counts.csv", index=False)


In [ ]:
# Use the results to analyze spots in the nuclei
spots_mapped, nuclei_analysis = analyze_spots(
    spots_df=df,
    nuclei_csv_path="/mnt/external.data/MeisterLab/mvolosko/image_project/sdc1v1/SDC1/1273/20240813_3d/nuclei/SDC1_3d_20240813_1273_E_late_15um_08.csv",
    mask_path=nuclei_mask_path 
)

# Analyse batch

In [ ]:
def process_folder(
    input_dir,
    output_dir,
    nuclei_mask_dir,
    nuclei_csv_dir,
    nuclei_prefix='SDC1_e_',
    mask_time_suffix='_t00',
    **detect_kwargs
):
    """
    Process all images in a folder for spot detection and nuclei analysis.

    Parameters:
    - input_dir: Directory containing input TIFF images
    - output_dir: Directory to save output CSVs
    - nuclei_mask_dir: Directory containing nuclei mask TIFFs
    - nuclei_csv_dir: Directory containing nuclei CSV files
    - nuclei_prefix: Prefix for nuclei-related filenames
    - mask_time_suffix: Time suffix for mask filenames (e.g., '_t00')
    - **detect_kwargs: Additional keyword arguments passed to detect_spots_MPHD
    """
    os.makedirs(output_dir, exist_ok=True)

    # Initialize empty summary DataFrame
    summary_df = pd.DataFrame(columns=[
        'image', 'total_spots', 'nuclei_data_available',
        'mean_spots_per_nuclei', 'median_spots_per_nuclei', 'total_nuclei',
        'processing_status', 'error_message'
    ])

    # Save path for summary file
    summary_path = os.path.join(output_dir, 'summary.csv')

    # Filter out *_max.tif* files and non-TIFF files
    image_files = [
        f for f in os.listdir(input_dir)
        if f.lower().endswith(('.tif', '.tiff'))
        and not os.path.splitext(f)[0].endswith('_max')
    ]

    for image_file in tqdm(image_files, desc="Processing images"):
        image_path = os.path.join(input_dir, image_file)
        base_name = os.path.splitext(image_file)[0]

        # Initialize row data
        row_data = {
            'image': image_file,
            'processing_status': 'started',
            'error_message': '',
            'total_spots': 0,
            'nuclei_data_available': False,
            'mean_spots_per_nuclei': None,
            'median_spots_per_nuclei': None,
            'total_nuclei': None
        }

        try:
            # Construct nuclei filenames based on naming convention
            nuclei_base = f"{nuclei_prefix}{base_name}"
            mask_filename = f"{nuclei_base}{mask_time_suffix}.tif"
            csv_filename = f"{nuclei_base}.csv"

            mask_path = os.path.join(nuclei_mask_dir, mask_filename)
            csv_path = os.path.join(nuclei_csv_dir, csv_filename)

            mask_exists = os.path.exists(mask_path)
            csv_exists = os.path.exists(csv_path)

            row_data['nuclei_data_available'] = mask_exists and csv_exists

            # Detect spots
            spots_df, _, _, _ = detect_spots_MPHD(
                image_path,
                nuclei_mask_path=mask_path if mask_exists else None,
                **detect_kwargs
            )

            row_data['total_spots'] = len(spots_df)

            # Save spots data
            spots_output = os.path.join(output_dir, f"{base_name}_spots.csv")
            spots_df.to_csv(spots_output, index=False)

            # Analyze nuclei if data exists
            if mask_exists and csv_exists:
                try:
                    nuclei_df = load_nuclei_data(csv_path)
                    spots_mapped, nuclei_metrics = analyze_spots(
                        spots_df, csv_path, mask_path, plot=False
                    )

                    metrics_output = os.path.join(output_dir, f"{base_name}_nuclei_metrics.csv")
                    nuclei_metrics.to_csv(metrics_output, index=False)

                    row_data.update({
                        'mean_spots_per_nuclei': nuclei_metrics['spot_count'].mean(),
                        'median_spots_per_nuclei': nuclei_metrics['spot_count'].median(),
                        'total_nuclei': len(nuclei_metrics)
                    })

                except Exception as e:
                    row_data['error_message'] = f"Nuclei analysis error: {str(e)}"

            row_data['processing_status'] = 'completed'

        except Exception as e:
            row_data['processing_status'] = 'failed'
            row_data['error_message'] = str(e)

        # Update summary DataFrame
        summary_df = pd.concat([
            summary_df,
            pd.DataFrame([row_data])
        ], ignore_index=True)

        # Save updated summary after each image
        summary_df.to_csv(summary_path, index=False)

    return summary_df




def analyze_spots(
    spots_csv_path,
    nuclei_csv_path,
    mask_path,
    output_dir=None,
    plot=False
):
    """
    Analyze spots in nuclei using saved CSV files
    
    Parameters:
    - spots_csv_path: Path to the CSV file containing spot coordinates
    - nuclei_csv_path: Path to the CSV file containing nuclei data
    - mask_path: Path to the nuclei mask file
    - output_dir: Directory to save output files (optional)
    - plot: Whether to generate and save plots (default: False)
    
    Returns:
    - Path to the mapped spots CSV file
    - Path to the nuclei metrics CSV file
    """
    try:
        # Load data from files
        spots_df = pd.read_csv(spots_csv_path)
        nuclei_df = load_nuclei_data(nuclei_csv_path)
        
        # Perform analysis
        spots_mapped, nuclei_counts = count_spots_in_nuclei(spots_df, mask_path, nuclei_df)
        
        # Calculate aspect ratios and filter
        nuclei_counts['xy_ratio'] = nuclei_counts.apply(
            lambda r: min(r['bb_dimX'], r['bb_dimY']) / max(r['bb_dimX'], r['bb_dimY']), axis=1)
        nuclei_counts['xz_ratio'] = nuclei_counts.apply(
            lambda r: min(r['bb_dimX'], r['bb_dimZ']) / max(r['bb_dimX'], r['bb_dimZ']), axis=1)
        nuclei_counts['yz_ratio'] = nuclei_counts.apply(
            lambda r: min(r['bb_dimY'], r['bb_dimZ']) / max(r['bb_dimY'], r['bb_dimZ']), axis=1)
        
        filtered_nuclei = nuclei_counts[
            (nuclei_counts['xy_ratio'] >= 0.5) &
            (nuclei_counts['xz_ratio'] >= 0.2) &
            (nuclei_counts['yz_ratio'] >= 0.2)
        ].copy()
        
        nuclei_metrics = calculate_spatial_metrics(filtered_nuclei)
        
        # Save results if output directory is provided
        if output_dir:
            os.makedirs(output_dir, exist_ok=True)
            
            # Generate output filenames
            base_name = os.path.splitext(os.path.basename(spots_csv_path))[0]
            spots_output = os.path.join(output_dir, f"{base_name}_mapped_spots.csv")
            metrics_output = os.path.join(output_dir, f"{base_name}_nuclei_metrics.csv")
            
            # Save results
            spots_mapped.to_csv(spots_output, index=False)
            nuclei_metrics.to_csv(metrics_output, index=False)
            
            # Generate and save plots if requested
            if plot:
                plot_output = os.path.join(output_dir, f"{base_name}_distribution.png")
                visualize_distribution(nuclei_metrics)
                plt.savefig(plot_output)
                plt.close()
            
            return spots_output, metrics_output
        else:
            return spots_mapped, nuclei_metrics


# RUN all

In [ ]:
process_folder(
    input_dir='/mnt/external.data/MeisterLab/Dario/SDC1/1273/20240813_e/N2V_sdc1_dpy27_mSG_emr1_mCh/denoised/',
    output_dir='/mnt/external.data/MeisterLab/mvolosko/image_project/sdc1/SDC1/1273/20240813_e/spots/',
    nuclei_mask_dir='/mnt/external.data/MeisterLab/mvolosko/image_project/sdc1v1/SDC1/1273/20240813_3d/segmentation/',
    nuclei_csv_dir='/mnt/external.data/MeisterLab/mvolosko/image_project/sdc1v1/SDC1/1273/20240813_3d/nuclei/',
    sigma=1,
    h_percentile=85
)
analyze_spots(spots_df, nuclei_csv_path='/mnt/external.data/MeisterLab/mvolosko/image_project/sdc1v1/SDC1/1273/20240813_3d/nuclei/',
 mask_path='/mnt/external.data/MeisterLab/mvolosko/image_project/sdc1v1/SDC1/1273/20240813_3d/segmentation/', plot=True)

# One use

In [ ]:
# Detect spots and visualize
df, img, hdome, labels = detect_spots_MPHD('path/to/single_image.tif')
visualize_results(img, df, hdome, labels)


# 3 D

In [12]:
def render_nuclei_3d(spots_df, mask, nuclei_df, step_size=2, level=0.5, z_range=None):
    """
    Modified 3D rendering with aspect ratio filtering
    """
    # Filter nuclei based on aspect ratios first
    filtered_nuclei = nuclei_df[
        (nuclei_df['xy_ratio'] >= 0.5) &
        (nuclei_df['xz_ratio'] >= 0.2) &
        (nuclei_df['yz_ratio'] >= 0.2)
    ]
    
    mask = mask.astype(np.uint16)
    labels_to_plot = filtered_nuclei['label'].tolist()

    if z_range is not None:
        z_start, z_end = z_range
        mask = mask[z_start:z_end]
        spots_df = spots_df[(spots_df['z_idx'] >= z_start) & (spots_df['z_idx'] < z_end)].copy()
        spots_df['z_idx'] -= z_start

    print(f"Rendering {len(labels_to_plot)} filtered nuclei...")

    fig = go.Figure()

    for label in labels_to_plot:
        binary_mask = (mask == label).astype(np.uint8)
        if np.count_nonzero(binary_mask) == 0:
            continue
        
        try:
            verts, faces, normals, values = marching_cubes(
                binary_mask, 
                level=level, 
                step_size=step_size
            )
        except RuntimeError:
            continue

        z, y, x = verts.T
        i, j, k = faces.T

        # Get matching nucleus data for coloring
        nucleus_data = filtered_nuclei[filtered_nuclei['label'] == label].iloc[0]
        
        fig.add_trace(go.Mesh3d(
            x=x, y=y, z=z,
            i=i, j=j, k=k,
            opacity=0.3,
            color='orange',
            name=f'Nucleus {label} (Spots: {nucleus_data["spot_count"]})',
            text=[
                f"Volume: {nucleus_data['volume']}<br>"
                f"Spots: {nucleus_data['spot_count']}<br>"
                f"XY Ratio: {nucleus_data['xy_ratio']:.2f}<br>"
                f"XZ Ratio: {nucleus_data['xz_ratio']:.2f}"
            ],
            hoverinfo='text',
            flatshading=True
        ))

    # Plot spots with same filtering
    filtered_spots = spots_df[spots_df['nucleus_label'].isin(labels_to_plot)]
    fig.add_trace(go.Scatter3d(
        x=filtered_spots['x_idx'],
        y=filtered_spots['y_idx'],
        z=filtered_spots['z_idx'],
        mode='markers',
        marker=dict(
            size=5,
            color=filtered_spots['nucleus_label'],
            colorscale='Viridis',
            opacity=0.6,
            colorbar=dict(title='Nucleus ID')
        ),
        name='Spots',
        hovertext=filtered_spots['label']
    ))

    fig.update_layout(
        title='3D Filtered Nuclei and Spots',
        scene=dict(
            xaxis_title='X', 
            yaxis_title='Y', 
            zaxis_title='Z',
            aspectmode='data'
        ),
        margin=dict(l=0, r=0, b=0, t=30)
    )
    
    fig.show()

In [ ]:
mask_path="/mnt/external.data/MeisterLab/mvolosko/image_project/sdc1v1/SDC1/1273/20240813_3d/segmentation/SDC1_3d_20240813_1273_E_late_15um_08_t00.tif"
nuclei_csv_path="/mnt/external.data/MeisterLab/mvolosko/image_project/sdc1v1/SDC1/1273/20240813_3d/nuclei/SDC1_3d_20240813_1273_E_late_15um_08.csv"
mask_img = bioio.BioImage(mask_path, reader=bioio_tifffile.Reader)
nuclei_mask = mask_img.get_image_data("ZYX", T=0, C=0)
nuclei_df = load_nuclei_data("/mnt/external.data/MeisterLab/mvolosko/image_project/sdc1v1/nuclei_with_spot_counts.csv")

render_nuclei_3d(spots_mapped, nuclei_mask, nuclei_analysis, z_range=(15, 40))#.sample(40).tolist())


# Previous tries

## LoG

In [ ]:

def detect_spots(image_path, spot_channel=1, 
                log_sigma=1.8, threshold_abs=0.2,
                min_distance=3, min_volume=8, max_volume=50,
                intensity_percentile=90):
    """
    Spot detection optimized for black background/white spots
    """
    # Load image directly without background subtraction
    img_4d = bioio.BioImage(image_path, reader=bioio_tifffile.Reader)
    img = img_4d.get_image_data("ZYX", T=0, C=spot_channel).astype(np.float32)

    # Direct LoG processing on raw image
    log = filters.gaussian(img, sigma=log_sigma)
    log = filters.laplace(log)
    log = -log  # Enhance bright spots

    # Adaptive threshold calculation
    non_zero_log = log[log > 0]
    if len(non_zero_log) > 0:
        adaptive_thresh = np.percentile(non_zero_log, 99)  # Focus on top 1% intensities
    else:
        adaptive_thresh = 0.1  # Fallback threshold

    # Find regional maxima
    coordinates = feature.peak_local_max(
        log,
        min_distance=min_distance,
        threshold_abs=adaptive_thresh,
        exclude_border=False
    )

    # Create markers
    mask = np.zeros_like(log, dtype=bool)
    mask[tuple(coordinates.T)] = True
    markers = measure.label(mask)

    # Watershed with dynamic mask
    mask_thresh = log > filters.threshold_otsu(log) if log.max() > 0 else np.zeros_like(log, dtype=bool)
    labels = segmentation.watershed(-log, markers, mask=mask_thresh)

    # Measure properties
    props = measure.regionprops_table(
        labels,
        intensity_image=img,  # Use original image for intensity measurements
        properties=('centroid', 'area', 'max_intensity')
    )

    df = pd.DataFrame(props).rename(columns={
        'centroid-0': 'z', 'centroid-1': 'y', 'centroid-2': 'x',
        'area': 'volume',
        'max_intensity': 'intensity'
    })
    
    # Intensity filtering based on original image
    intensity_thresh = np.percentile(img[img > 0], intensity_percentile)  # Focus on non-zero pixels
    df = df[
        (df['volume'].between(min_volume, max_volume)) &
        (df['intensity'] > intensity_thresh)
    ]

    return df, img, log, labels

def visualize_results(img, df, log, labels):
    """Visualization focusing on original image characteristics"""
    max_proj = np.max(img, axis=0)
    log_proj = np.max(log, axis=0)

      # Find middle Z-slice and its detections
    mid_z = img.shape[0] // 2
    df_mid = df[df['z'].round().astype(int) == mid_z]  # Match spots to slice
    
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 3, 1)
    plt.imshow(max_proj, cmap='gray', vmax=np.percentile(max_proj, 99.5))
    plt.scatter(df['x'], df['y'], s=30, facecolors='none', marker='o', 
                edgecolors='magenta', linewidth=0.8, alpha=0.8)
    plt.title(f"Detected spots: {len(df)}")
    
    plt.subplot(1, 3, 2)
    plt.imshow(log_proj, cmap='viridis')#, vmax=np.percentile(log_proj, 99.5))
    plt.contour(log_proj > np.percentile(log, 99.5), colors='white', alpha=0.3)
    plt.title("Processed LoG Image")
    
    # Z-slice validation
    plt.subplot(1, 3, 3)
    plt.imshow(img[mid_z], cmap='gray')
    plt.scatter(df_mid['x'], df_mid['y'], s=60, facecolors='none',
                marker='o', edgecolors='red', linewidth=0.8)
    plt.title(f"Z={mid_z} Validation: {len(df_mid)} spots")
    
    plt.tight_layout()
    plt.show()

# Usage example
if __name__ == "__main__":
    df, img, log, labels = detect_spots(
        image_path="/mnt/external.data/MeisterLab/Dario/SDC1/1273/20240813_3d/N2V_sdc1_dpy27_mSG_emr1_mCh/denoised/20240813_1273_E_late_15um_08_n2v.tif",
        spot_channel=1,
        log_sigma=0.001,             # Adjust based on actual spot size
        threshold_abs=0.2,         # Start with 0.1-0.3 range
        min_distance=1,            # Physical separation between spots
        min_volume=10,             # Minimum voxels per spot
        max_volume=50,             # Maximum voxels per spot
        intensity_percentile=99.5    # Strong intensity cutoff
    )
    visualize_results(img, df, log, labels)

In [ ]:
def detect_spots(image_path, spot_channel=1, sigma_xy=1.0, sigma_z=0.5,
                 threshold_percentile=99, min_volume=10):
    """
    Detect spots in a 3D z-stack TIFF image.
    """
    # Load image with proper dimension handling
    img_4d = bioio.BioImage(image_path, reader=bioio_tifffile.Reader)
    img = img_4d.get_image_data("ZYX", T=0, C=spot_channel)  # Direct 3D array

    # Preprocessing: 3D Gaussian blur
    blurred = filters.gaussian(
        img,
        sigma=(sigma_z, sigma_xy, sigma_xy),
        mode='nearest'
    )

    # Thresholding
    threshold = np.percentile(blurred, threshold_percentile)
    binary_mask = blurred > threshold

    # Remove small objects
    cleaned_mask = morphology.remove_small_objects(binary_mask, min_size=min_volume)

    # Label regions and measure properties
    label_image = measure.label(cleaned_mask)
    props = measure.regionprops_table(
        label_image,
        intensity_image=blurred,
        properties=('centroid', 'area', 'mean_intensity')
    )
    
    # Create DataFrame
    df = pd.DataFrame(props)
    df = df.rename(columns={
        'centroid-0': 'z',
        'centroid-1': 'y',
        'centroid-2': 'x',
        'area': 'volume',
        'mean_intensity': 'intensity'
    })

    return df, img

def visualize_results(img, df):
    """Plot maximum projection with detected spots and middle Z-slice."""
    max_proj = np.max(img, axis=0)
    mid_z = img.shape[0] // 2

    # Find spots within ±0.5 of middle Z-slice (strict middle plane)
    df_mid = df[(df['z'] >= mid_z - 0.5) & (df['z'] < mid_z + 0.5)]

    plt.figure(figsize=(12, 5))
    
    # Max projection
    plt.subplot(1, 2, 1)
    plt.imshow(max_proj, cmap='gray')
    plt.scatter(df['x'], df['y'], s=40, facecolors='none', marker='o', 
                edgecolors='red', linewidths=0.5)
    plt.title(f"Max Projection: {len(df)} spots")
    
    # Middle slice
    plt.subplot(1, 2, 2)
    plt.imshow(img[mid_z], cmap='gray')
    plt.scatter(df_mid['x'], df_mid['y'], s=40, facecolors='none', marker='o', 
                edgecolors='red', linewidths=0.5)
    plt.title(f"Z={mid_z} Slice: {len(df_mid)} spots")
    
    plt.tight_layout()
    plt.show()

# Example usage
if __name__ == "__main__":
    df, img = detect_spots(
        image_path="/mnt/external.data/MeisterLab/Dario/SDC1/1273/20240813_3d/N2V_sdc1_dpy27_mSG_emr1_mCh/denoised/20240813_1273_E_late_15um_08_n2v.tif",
        spot_channel=1,
        sigma_xy=0,
        sigma_z=0,
        threshold_percentile=99,
        min_volume=0
    )
    visualize_results(img, df)
    print(df.head())


## Spots with background

In [ ]:
def detect_spots_MPHD(image_path, spot_channel=1, 
                     sigma=1.8, h_percentile=85,  
                     min_distance=2,              
                     min_volume=5,                
                     max_volume=100, 
                     intensity_percentile=80,     
                     peak_percentile=90):         
    """
    MPHD spot detection with adaptive h-dome transformation.
    Parameters:
    - image_path: Path to the 3D TIFF image.
    - spot_channel: Channel index for spot detection.
    - sigma: Gaussian smoothing parameter.
    - h_percentile: Percentile for adaptive h value.
    - min_distance: Minimum distance between detected spots.
    - min_volume: Minimum volume of detected spots.
    - max_volume: Maximum volume of detected spots.
    - intensity_percentile: Percentile for intensity filtering.
    - peak_percentile: Percentile for peak thresholding.
    """
    # Load image with intensity scaling
    img_4d = bioio.BioImage(image_path, reader=bioio_tifffile.Reader)
    img_raw = img_4d.get_image_data("ZYX", T=0, C=spot_channel).astype(np.float32)
    
    # Normalize to [0,1] based on image statistics
    img = (img_raw - img_raw.min()) / (img_raw.max() - img_raw.min() + 1e-6)
    
    # Diagnostic output
    print(f"Image intensity stats - Min: {img.min():.3f}, Max: {img.max():.3f}, Mean: {img.mean():.3f}")
    
    # Gaussian smoothing with 3D kernel
    smoothed = filters.gaussian(img, sigma=sigma)
    
    # Calculate adaptive h value
    non_zero = smoothed[smoothed > 0]
    if len(non_zero) == 0:
        raise ValueError("No signal detected after smoothing!")
        
    h = np.percentile(non_zero, h_percentile)
    print(f"Calculated h-value: {h:.3f} (percentile {h_percentile})")
    
    # 3D h-dome transformation
    seed = np.clip(smoothed - h, 0, None)
    footprint = morphology.ball(1)
    reconstructed = morphology.reconstruction(seed, smoothed, method='dilation', footprint=footprint)
    hdome = smoothed - reconstructed
    
    # Diagnostic visualization
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 3, 1)
    plt.imshow(np.max(smoothed, axis=0), cmap='gray')
    plt.title("Smoothed Image")
    
    plt.subplot(1, 3, 2)
    plt.imshow(np.max(seed, axis=0), cmap='gray')
    plt.title(f"Seed Image (h={h:.3f})")
    
    plt.subplot(1, 3, 3)
    plt.imshow(np.max(hdome, axis=0), cmap='viridis')
    plt.title("H-Dome Projection")
    plt.show()
    
    # Adaptive peak threshold
    non_zero_hdome = hdome[hdome > 0]
    if len(non_zero_hdome) > 0:
        peak_thresh = np.percentile(non_zero_hdome, peak_percentile)
        print(f"Peak threshold: {peak_thresh:.3f} (percentile {peak_percentile})")
    else:
        raise ValueError("H-dome transformation failed - check h value!")
    
    # Detect regional maxima with relaxed thresholds
    coordinates = feature.peak_local_max(
        hdome,
        min_distance=min_distance,
        threshold_abs=peak_thresh * 0.8,  # 20% margin
        exclude_border=False
    )
    
    print(f"Initial candidates: {len(coordinates)} spots")
    
    peak_df = pd.DataFrame(coordinates, columns=['peak_z', 'peak_y', 'peak_x'])
    
    # Create markers
    mask = np.zeros_like(hdome, dtype=bool)
    mask[tuple(coordinates.T)] = True
    markers = measure.label(mask)

    # Watershed segmentation with Otsu masking
    if hdome.max() > 0:
        mask_thresh = hdome > filters.threshold_otsu(hdome)
    else:
        mask_thresh = np.zeros_like(hdome, dtype=bool)
    
    labels = segmentation.watershed(-hdome, markers, mask=mask_thresh)

    # Measure properties including original label IDs
    props = measure.regionprops_table(
        labels,
        intensity_image=img,
        properties=('label', 'centroid', 'area', 'max_intensity')
    )
    
    df = pd.DataFrame(props).rename(columns={
        'centroid-0': 'centroid_z', 'centroid-1': 'y', 'centroid-2': 'x',
        'area': 'volume',
        'max_intensity': 'intensity'
    })
    
    # Add peak coordinates using label mapping
    peak_df['label'] = markers[tuple(coordinates.T)]  # Get label for each peak
    df = df.merge(peak_df, on='label', how='left')

    # Intensity filtering
    if len(img[img > 0]) > 0:
        intensity_thresh = np.percentile(img[img > 0], intensity_percentile)
        df = df[df['intensity'] > intensity_thresh]
    
    # Volume filtering
    df = df[df['volume'].between(min_volume, max_volume)]

    return df, img, hdome, labels


def visualize_results(img, df, hdome, labels):
    """Visualization for MPHD results"""
    max_proj = np.max(img, axis=0)
    hdome_proj = np.max(hdome, axis=0)
    mid_z = img.shape[0] // 2
    
    # Filter by PEAK Z-coordinate
    df_mid = df[df['peak_z'] == mid_z]

    plt.figure(figsize=(10, 5))
    
    plt.subplot(1, 2, 1)
    plt.imshow(max_proj, cmap='gray', vmax=np.percentile(max_proj, 99.5))
    plt.scatter(df['peak_x'], df['peak_y'], s=20, facecolors='none', 
                edgecolors='red', linewidth=0.5)
    plt.title(f"Detected spots: {len(df)}")
    
    plt.subplot(1, 2, 2)
    plt.imshow(hdome_proj, cmap='viridis')
    plt.title("H-Dome Projection")
    

# # Usage with recommended starting parameters
# if __name__ == "__main__":
#     df, img, hdome, labels = detect_spots_MPHD(
#         #image_path="/mnt/external.data/MeisterLab/Dario/SDC1/1273/20240813_3d/N2V_sdc1_dpy27_mSG_emr1_mCh/denoised/20240813_1273_E_late_15um_08_n2v.tif",
#         image_path="/mnt/external.data/MeisterLab/Dario/DPY27/1268/20240808_3d/N2V_dpy27_mSG_emr1_mCh/denoised/20240808_1268_E_late_15um_08_n2v.tif",
#         spot_channel=1,
#         sigma=1,               # Start with smaller sigma
#         h_percentile=70,         # Lower h-percentile
#         peak_percentile=75,      # Lower peak threshold
#         min_distance=2,          # Reduced separation
#         min_volume=5,            # Smaller volume
#         max_volume=150,
#         intensity_percentile=80  # More lenient intensity cutoff
#     )
#     visualize_results(img, df, hdome, labels)


# 3 d old

In [ ]:
def plot_3d_spots_and_nuclei(spots_df, nuclei_df, color_by='nucleus_label'):
    """
    Creates an interactive 3D scatter plot of spot and nuclei data.

    :param spots_df: DataFrame with 'x_idx', 'y_idx', 'z_idx', and optionally 'nucleus_label'.
    :param nuclei_df: DataFrame with 'centroid_x', 'centroid_y', 'centroid_z'.
    :param color_by: Column in spots_df to color by (e.g., 'nucleus_label' or 'intensity').
    """
    fig = go.Figure()

    # Plot spots
    fig.add_trace(go.Scatter3d(
        x=spots_df['x_idx'],
        y=spots_df['y_idx'],
        z=spots_df['z_idx'],
        mode='markers',
        marker=dict(
            size=6,
            color=spots_df[color_by] if color_by in spots_df.columns else 'red',
            colorscale='Viridis',
            opacity=0.6,
            colorbar=dict(title=color_by)
        ),
        name='Spots'
    ))

    # Plot nuclei centroids
    if all(f'centroid_{ax}' in nuclei_df.columns for ax in ['x', 'y', 'z']):
        fig.add_trace(go.Scatter3d(
            x=nuclei_df['centroid_x'],
            y=nuclei_df['centroid_y'],
            z=nuclei_df['centroid_z'],
            mode='markers',
            marker=dict(
                size=15,
                color='orange',
                opacity=0.3,
                symbol='circle'
            ),
            name='Nuclei Centers'
        ))

    fig.update_layout(
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z',
            aspectmode='data',
        ),
        title='3D Visualization of Spots and Nuclei',
        height=800
    )

    fig.show()

#spots_mapped, nuclei_metrics = analyze_spots(spots_df, nuclei_csv_path, mask_path)

# Call the 3D plot function
plot_3d_spots_and_nuclei(spots_mapped, nuclei_analysis, color_by='nucleus_label')